In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential, load_model
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras.utils import to_categorical
from sklearn import metrics
from sklearn.model_selection import train_test_split
import h5py
from sklearn import utils
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from numpy.random import randint

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = keras.datasets.mnist.load_data()


11493376/11490434 [==============================] - 0s 0us/step


In [8]:
wrong_frac = 0.00
version = 9
name_frac = str(wrong_frac).split('.')[-1]
if len(name_frac) == 1:
    name_frac = name_frac + '0'
save_weights = '/content/drive/My Drive/colab/'+ name_frac +'perror_weights_v'+ str(version) + '.h5'
save_history = '/content/drive/My Drive/colab/'+ name_frac +'perror_history_v'+ str(version) + '.h5'


seven_imgs_train = []
seven_labels_train = []
for idx,label in enumerate(data[0][1]):
    if label == 7:
        seven_imgs_train.append(data[0][0][idx])
        seven_labels_train.append(label)
seven_imgs_train = np.array(seven_imgs_train)
seven_labels_train = np.array(seven_labels_train)


new_left_images = []
new_left_labels = []
new_right_images = []
new_right_labels = []

for img in seven_imgs_train:
    new_left_images.append(np.fliplr(img))
    new_left_labels.append(0)
    new_right_images.append(img)
    new_right_labels.append(1)
new_left_images = np.array(new_left_images)
new_right_images = np.array(new_right_images)


X = utils.shuffle(np.concatenate((new_right_images,new_left_images),axis =0),random_state=0)
Y = utils.shuffle(np.concatenate((new_right_labels,new_left_labels),axis = 0),random_state=0)

train_split = int(len(X)*0.8)
val_split = train_split+int(len(X)*0.1)

X_train = X[:train_split]
X_test = X[train_split:val_split]
# X_val = X[val_split:]
Y_train = Y[:train_split]
Y_test = Y[train_split:val_split]
# X_val = X[val_split:]


X_train_norm = X_train/X_train.max()
X_test_norm = X_test/X_test.max()
X_train_norm = np.expand_dims(X_train_norm,axis=3)
X_test_norm = np.expand_dims(X_test_norm,axis=3)


batch_size = 32
seed = 42



train_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)

test_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)



train_generator = train_datagen.flow(X_train_norm, y=Y_train, batch_size=batch_size,seed=seed)
val_generator = test_datagen.flow(X_test_norm,y=Y_test,batch_size=batch_size,seed=seed)


modelE = keras.models.Sequential()
modelE.add(Conv2D(64, (3, 3), input_shape=(28, 28,1)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))


modelE.add(Conv2D(64, (3, 3)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))

modelE.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
modelE.add(Dense(64))
modelE.add(Activation('relu'))
modelE.add(Dropout(0.5))
modelE.add(Dense(1))
modelE.add(Activation('sigmoid'))

modelE.compile(loss='binary_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])


modelCheckpoint = ModelCheckpoint(save_weights,
                                  monitor = 'val_loss',
                                  save_best_only = True,
                                  mode = 'min',
                                  verbose = 2,
                                  save_weights_only = True)
callbacks_list = [modelCheckpoint]
history = modelE.fit_generator(
        train_generator,
        steps_per_epoch=2870,
        epochs=100,
        validation_data=val_generator,
        validation_steps=358,
        verbose = 0,
        callbacks=callbacks_list)
modelE.save_weights(save_weights)
h = h5py.File(save_history,'w')
h_keys = history.history.keys()
for k in h_keys:
    h.create_dataset(k,data=history.history[k])
h.close()

seven_imgs_test = []
for idx,label in enumerate(data[1][1]):
    if label == 7:
        im_max =data[1][0][idx].max()
        seven_imgs_test.append(data[1][0][idx]/im_max)
seven_imgs_test = np.array(seven_imgs_test)

X_left = np.array([np.fliplr(img) for img in seven_imgs_test])
X = np.expand_dims(np.concatenate((seven_imgs_test,X_left),axis=0),axis=3)
Y = np.concatenate((np.ones((seven_imgs_test.shape[0],)),np.zeros((X_left.shape[0],))),axis=0)

print(modelE.evaluate(X,Y))


Epoch 00001: val_loss improved from inf to 0.58406, saving model to /content/drive/My Drive/colab/00perror_weights_v9.h5

Epoch 00002: val_loss improved from 0.58406 to 0.40213, saving model to /content/drive/My Drive/colab/00perror_weights_v9.h5

Epoch 00003: val_loss improved from 0.40213 to 0.21518, saving model to /content/drive/My Drive/colab/00perror_weights_v9.h5

Epoch 00004: val_loss improved from 0.21518 to 0.11440, saving model to /content/drive/My Drive/colab/00perror_weights_v9.h5

Epoch 00005: val_loss improved from 0.11440 to 0.07167, saving model to /content/drive/My Drive/colab/00perror_weights_v9.h5

Epoch 00006: val_loss improved from 0.07167 to 0.05391, saving model to /content/drive/My Drive/colab/00perror_weights_v9.h5

Epoch 00007: val_loss improved from 0.05391 to 0.04392, saving model to /content/drive/My Drive/colab/00perror_weights_v9.h5

Epoch 00008: val_loss improved from 0.04392 to 0.03947, saving model to /content/drive/My Drive/colab/00perror_weights_v9

In [6]:
wrong_frac = 0.05
version = 9
name_frac = str(wrong_frac).split('.')[-1]
if len(name_frac) == 1:
    name_frac = name_frac + '0'
if len(name_frac) == 1:
    name_frac = name_frac + '0'
save_weights = '/content/drive/My Drive/colab/'+ name_frac +'perror_weights_v'+ str(version) + '.h5'
save_history = '/content/drive/My Drive/colab/'+ name_frac +'perror_history_v'+ str(version) + '.h5'


seven_imgs_train = []
seven_labels_train = []
for idx,label in enumerate(data[0][1]):
    if label == 7:
        seven_imgs_train.append(data[0][0][idx])
        seven_labels_train.append(label)
seven_imgs_train = np.array(seven_imgs_train)
seven_labels_train = np.array(seven_labels_train)


new_left_images = []
new_left_labels = []
new_right_images = []
new_right_labels = []

for img in seven_imgs_train:
    new_left_images.append(np.fliplr(img))
    new_left_labels.append(0)
    new_right_images.append(img)
    new_right_labels.append(1)
new_left_images = np.array(new_left_images)
new_right_images = np.array(new_right_images)

split = int(seven_imgs_train.shape[0]*wrong_frac)

for idx in np.arange(0,split):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(0,split):
    new_left_images[idx] = np.fliplr(new_left_images[idx])


X = utils.shuffle(np.concatenate((new_right_images,new_left_images),axis =0),random_state=0)
Y = utils.shuffle(np.concatenate((new_right_labels,new_left_labels),axis = 0),random_state=0)

train_split = int(len(X)*0.8)
val_split = train_split+int(len(X)*0.1)

X_train = X[:train_split]
X_test = X[train_split:val_split]
# X_val = X[val_split:]
Y_train = Y[:train_split]
Y_test = Y[train_split:val_split]
# X_val = X[val_split:]

X_train_norm = X_train/X_train.max()
X_test_norm = X_test/X_test.max()
X_train_norm = np.expand_dims(X_train_norm,axis=3)
X_test_norm = np.expand_dims(X_test_norm,axis=3)


batch_size = 32
seed = 42

def random_180(img):
    turns = randint(0,2)
    turns  = turns*2
    return np.rot90(img,turns)

train_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)

test_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)



train_generator = train_datagen.flow(X_train_norm, y=Y_train, batch_size=batch_size,seed=seed)
val_generator = test_datagen.flow(X_test_norm,y=Y_test,batch_size=batch_size,seed=seed)


modelE = keras.models.Sequential()
modelE.add(Conv2D(64, (3, 3), input_shape=(28, 28,1)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))


modelE.add(Conv2D(64, (3, 3)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))

modelE.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
modelE.add(Dense(64))
modelE.add(Activation('relu'))
modelE.add(Dropout(0.5))
modelE.add(Dense(1))
modelE.add(Activation('sigmoid'))

modelE.compile(loss='binary_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])


modelCheckpoint = ModelCheckpoint(save_weights,
                                  monitor = 'val_loss',
                                  save_best_only = True,
                                  mode = 'min',
                                  verbose = 2,
                                  save_weights_only = True)
callbacks_list = [modelCheckpoint]
history = modelE.fit_generator(
        train_generator,
        steps_per_epoch=2870,
        epochs=100,
        validation_data=val_generator,
        validation_steps=358,
        verbose = 0,
        callbacks=callbacks_list)
modelE.save_weights(save_weights)
h = h5py.File(save_history,'w')
h_keys = history.history.keys()
for k in h_keys:
    h.create_dataset(k,data=history.history[k])
h.close()

print(modelE.evaluate(X_test_norm,Y_test))


Epoch 00001: val_loss improved from inf to 0.58278, saving model to /content/drive/My Drive/colab/05perror_weights_v9.h5

Epoch 00002: val_loss improved from 0.58278 to 0.43880, saving model to /content/drive/My Drive/colab/05perror_weights_v9.h5

Epoch 00003: val_loss improved from 0.43880 to 0.33524, saving model to /content/drive/My Drive/colab/05perror_weights_v9.h5

Epoch 00004: val_loss improved from 0.33524 to 0.28937, saving model to /content/drive/My Drive/colab/05perror_weights_v9.h5

Epoch 00005: val_loss improved from 0.28937 to 0.27768, saving model to /content/drive/My Drive/colab/05perror_weights_v9.h5

Epoch 00006: val_loss improved from 0.27768 to 0.27681, saving model to /content/drive/My Drive/colab/05perror_weights_v9.h5

Epoch 00007: val_loss improved from 0.27681 to 0.27279, saving model to /content/drive/My Drive/colab/05perror_weights_v9.h5

Epoch 00008: val_loss improved from 0.27279 to 0.27052, saving model to /content/drive/My Drive/colab/05perror_weights_v9

In [7]:
wrong_frac = 0.1
version = 9
name_frac = str(wrong_frac).split('.')[-1]
if len(name_frac) == 1:
    name_frac = name_frac + '0'
if len(name_frac) == 1:
    name_frac = name_frac + '0'
save_weights = '/content/drive/My Drive/colab/'+ name_frac +'perror_weights_v'+ str(version) + '.h5'
save_history = '/content/drive/My Drive/colab/'+ name_frac +'perror_history_v'+ str(version) + '.h5'


seven_imgs_train = []
seven_labels_train = []
for idx,label in enumerate(data[0][1]):
    if label == 7:
        seven_imgs_train.append(data[0][0][idx])
        seven_labels_train.append(label)
seven_imgs_train = np.array(seven_imgs_train)
seven_labels_train = np.array(seven_labels_train)


new_left_images = []
new_left_labels = []
new_right_images = []
new_right_labels = []

for img in seven_imgs_train:
    new_left_images.append(np.fliplr(img))
    new_left_labels.append(0)
    new_right_images.append(img)
    new_right_labels.append(1)
new_left_images = np.array(new_left_images)
new_right_images = np.array(new_right_images)

split = int(seven_imgs_train.shape[0]*wrong_frac)

for idx in np.arange(0,split):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(0,split):
    new_left_images[idx] = np.fliplr(new_left_images[idx])


X = utils.shuffle(np.concatenate((new_right_images,new_left_images),axis =0),random_state=0)
Y = utils.shuffle(np.concatenate((new_right_labels,new_left_labels),axis = 0),random_state=0)

train_split = int(len(X)*0.8)
val_split = train_split+int(len(X)*0.1)

X_train = X[:train_split]
X_test = X[train_split:val_split]
# X_val = X[val_split:]
Y_train = Y[:train_split]
Y_test = Y[train_split:val_split]
# X_val = X[val_split:]

X_train_norm = X_train/X_train.max()
X_test_norm = X_test/X_test.max()
X_train_norm = np.expand_dims(X_train_norm,axis=3)
X_test_norm = np.expand_dims(X_test_norm,axis=3)


batch_size = 32
seed = 42

def random_180(img):
    turns = randint(0,2)
    turns  = turns*2
    return np.rot90(img,turns)

train_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)

test_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)



train_generator = train_datagen.flow(X_train_norm, y=Y_train, batch_size=batch_size,seed=seed)
val_generator = test_datagen.flow(X_test_norm,y=Y_test,batch_size=batch_size,seed=seed)


modelE = keras.models.Sequential()
modelE.add(Conv2D(64, (3, 3), input_shape=(28, 28,1)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))


modelE.add(Conv2D(64, (3, 3)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))

modelE.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
modelE.add(Dense(64))
modelE.add(Activation('relu'))
modelE.add(Dropout(0.5))
modelE.add(Dense(1))
modelE.add(Activation('sigmoid'))

modelE.compile(loss='binary_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])


modelCheckpoint = ModelCheckpoint(save_weights,
                                  monitor = 'val_loss',
                                  save_best_only = True,
                                  mode = 'min',
                                  verbose = 2,
                                  save_weights_only = True)
callbacks_list = [modelCheckpoint]
history = modelE.fit_generator(
        train_generator,
        steps_per_epoch=2870,
        epochs=100,
        validation_data=val_generator,
        validation_steps=358,
        verbose = 0,
        callbacks=callbacks_list)
modelE.save_weights(save_weights)
h = h5py.File(save_history,'w')
h_keys = history.history.keys()
for k in h_keys:
    h.create_dataset(k,data=history.history[k])
h.close()

print(modelE.evaluate(X_test_norm,Y_test))


Epoch 00001: val_loss improved from inf to 0.62298, saving model to /content/drive/My Drive/colab/10perror_weights_v9.h5

Epoch 00002: val_loss improved from 0.62298 to 0.53581, saving model to /content/drive/My Drive/colab/10perror_weights_v9.h5

Epoch 00003: val_loss improved from 0.53581 to 0.45236, saving model to /content/drive/My Drive/colab/10perror_weights_v9.h5

Epoch 00004: val_loss improved from 0.45236 to 0.40604, saving model to /content/drive/My Drive/colab/10perror_weights_v9.h5

Epoch 00005: val_loss improved from 0.40604 to 0.39003, saving model to /content/drive/My Drive/colab/10perror_weights_v9.h5

Epoch 00006: val_loss improved from 0.39003 to 0.38891, saving model to /content/drive/My Drive/colab/10perror_weights_v9.h5

Epoch 00007: val_loss improved from 0.38891 to 0.38416, saving model to /content/drive/My Drive/colab/10perror_weights_v9.h5

Epoch 00008: val_loss improved from 0.38416 to 0.38115, saving model to /content/drive/My Drive/colab/10perror_weights_v9

In [8]:
wrong_frac = 0.2
version = 9
name_frac = str(wrong_frac).split('.')[-1]
if len(name_frac) == 1:
    name_frac = name_frac + '0'
if len(name_frac) == 1:
    name_frac = name_frac + '0'
save_weights = '/content/drive/My Drive/colab/'+ name_frac +'perror_weights_v'+ str(version) + '.h5'
save_history = '/content/drive/My Drive/colab/'+ name_frac +'perror_history_v'+ str(version) + '.h5'


seven_imgs_train = []
seven_labels_train = []
for idx,label in enumerate(data[0][1]):
    if label == 7:
        seven_imgs_train.append(data[0][0][idx])
        seven_labels_train.append(label)
seven_imgs_train = np.array(seven_imgs_train)
seven_labels_train = np.array(seven_labels_train)


new_left_images = []
new_left_labels = []
new_right_images = []
new_right_labels = []

for img in seven_imgs_train:
    new_left_images.append(np.fliplr(img))
    new_left_labels.append(0)
    new_right_images.append(img)
    new_right_labels.append(1)
new_left_images = np.array(new_left_images)
new_right_images = np.array(new_right_images)

split = int(seven_imgs_train.shape[0]*wrong_frac)

for idx in np.arange(0,split):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(0,split):
    new_left_images[idx] = np.fliplr(new_left_images[idx])


X = utils.shuffle(np.concatenate((new_right_images,new_left_images),axis =0),random_state=0)
Y = utils.shuffle(np.concatenate((new_right_labels,new_left_labels),axis = 0),random_state=0)

train_split = int(len(X)*0.8)
val_split = train_split+int(len(X)*0.1)

X_train = X[:train_split]
X_test = X[train_split:val_split]
# X_val = X[val_split:]
Y_train = Y[:train_split]
Y_test = Y[train_split:val_split]
# X_val = X[val_split:]

X_train_norm = X_train/X_train.max()
X_test_norm = X_test/X_test.max()
X_train_norm = np.expand_dims(X_train_norm,axis=3)
X_test_norm = np.expand_dims(X_test_norm,axis=3)


batch_size = 32
seed = 42

def random_180(img):
    turns = randint(0,2)
    turns  = turns*2
    return np.rot90(img,turns)

train_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)

test_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)



train_generator = train_datagen.flow(X_train_norm, y=Y_train, batch_size=batch_size,seed=seed)
val_generator = test_datagen.flow(X_test_norm,y=Y_test,batch_size=batch_size,seed=seed)


modelE = keras.models.Sequential()
modelE.add(Conv2D(64, (3, 3), input_shape=(28, 28,1)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))


modelE.add(Conv2D(64, (3, 3)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))

modelE.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
modelE.add(Dense(64))
modelE.add(Activation('relu'))
modelE.add(Dropout(0.5))
modelE.add(Dense(1))
modelE.add(Activation('sigmoid'))

modelE.compile(loss='binary_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])


modelCheckpoint = ModelCheckpoint(save_weights,
                                  monitor = 'val_loss',
                                  save_best_only = True,
                                  mode = 'min',
                                  verbose = 2,
                                  save_weights_only = True)
callbacks_list = [modelCheckpoint]
history = modelE.fit_generator(
        train_generator,
        steps_per_epoch=2870,
        epochs=100,
        validation_data=val_generator,
        validation_steps=358,
        verbose = 0,
        callbacks=callbacks_list)
modelE.save_weights(save_weights)
h = h5py.File(save_history,'w')
h_keys = history.history.keys()
for k in h_keys:
    h.create_dataset(k,data=history.history[k])
h.close()

print(modelE.evaluate(X_test_norm,Y_test))


Epoch 00001: val_loss improved from inf to 0.65791, saving model to /content/drive/My Drive/colab/20perror_weights_v9.h5

Epoch 00002: val_loss improved from 0.65791 to 0.62045, saving model to /content/drive/My Drive/colab/20perror_weights_v9.h5

Epoch 00003: val_loss improved from 0.62045 to 0.58465, saving model to /content/drive/My Drive/colab/20perror_weights_v9.h5

Epoch 00004: val_loss improved from 0.58465 to 0.55650, saving model to /content/drive/My Drive/colab/20perror_weights_v9.h5

Epoch 00005: val_loss improved from 0.55650 to 0.54250, saving model to /content/drive/My Drive/colab/20perror_weights_v9.h5

Epoch 00006: val_loss improved from 0.54250 to 0.54006, saving model to /content/drive/My Drive/colab/20perror_weights_v9.h5

Epoch 00007: val_loss improved from 0.54006 to 0.53646, saving model to /content/drive/My Drive/colab/20perror_weights_v9.h5

Epoch 00008: val_loss improved from 0.53646 to 0.53127, saving model to /content/drive/My Drive/colab/20perror_weights_v9

In [9]:
wrong_frac = 0.3
version = 9
name_frac = str(wrong_frac).split('.')[-1]
if len(name_frac) == 1:
    name_frac = name_frac + '0'
if len(name_frac) == 1:
    name_frac = name_frac + '0'
save_weights = '/content/drive/My Drive/colab/'+ name_frac +'perror_weights_v'+ str(version) + '.h5'
save_history = '/content/drive/My Drive/colab/'+ name_frac +'perror_history_v'+ str(version) + '.h5'


seven_imgs_train = []
seven_labels_train = []
for idx,label in enumerate(data[0][1]):
    if label == 7:
        seven_imgs_train.append(data[0][0][idx])
        seven_labels_train.append(label)
seven_imgs_train = np.array(seven_imgs_train)
seven_labels_train = np.array(seven_labels_train)


new_left_images = []
new_left_labels = []
new_right_images = []
new_right_labels = []

for img in seven_imgs_train:
    new_left_images.append(np.fliplr(img))
    new_left_labels.append(0)
    new_right_images.append(img)
    new_right_labels.append(1)
new_left_images = np.array(new_left_images)
new_right_images = np.array(new_right_images)

split = int(seven_imgs_train.shape[0]*wrong_frac)

for idx in np.arange(0,split):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(0,split):
    new_left_images[idx] = np.fliplr(new_left_images[idx])


X = utils.shuffle(np.concatenate((new_right_images,new_left_images),axis =0),random_state=0)
Y = utils.shuffle(np.concatenate((new_right_labels,new_left_labels),axis = 0),random_state=0)

train_split = int(len(X)*0.8)
val_split = train_split+int(len(X)*0.1)

X_train = X[:train_split]
X_test = X[train_split:val_split]
# X_val = X[val_split:]
Y_train = Y[:train_split]
Y_test = Y[train_split:val_split]
# X_val = X[val_split:]

X_train_norm = X_train/X_train.max()
X_test_norm = X_test/X_test.max()
X_train_norm = np.expand_dims(X_train_norm,axis=3)
X_test_norm = np.expand_dims(X_test_norm,axis=3)


batch_size = 32
seed = 42

def random_180(img):
    turns = randint(0,2)
    turns  = turns*2
    return np.rot90(img,turns)

train_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)

test_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)



train_generator = train_datagen.flow(X_train_norm, y=Y_train, batch_size=batch_size,seed=seed)
val_generator = test_datagen.flow(X_test_norm,y=Y_test,batch_size=batch_size,seed=seed)


modelE = keras.models.Sequential()
modelE.add(Conv2D(64, (3, 3), input_shape=(28, 28,1)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))


modelE.add(Conv2D(64, (3, 3)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))

modelE.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
modelE.add(Dense(64))
modelE.add(Activation('relu'))
modelE.add(Dropout(0.5))
modelE.add(Dense(1))
modelE.add(Activation('sigmoid'))

modelE.compile(loss='binary_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])


modelCheckpoint = ModelCheckpoint(save_weights,
                                  monitor = 'val_loss',
                                  save_best_only = True,
                                  mode = 'min',
                                  verbose = 2,
                                  save_weights_only = True)
callbacks_list = [modelCheckpoint]
history = modelE.fit_generator(
        train_generator,
        steps_per_epoch=2870,
        epochs=100,
        validation_data=val_generator,
        validation_steps=358,
        verbose = 0,
        callbacks=callbacks_list)
modelE.save_weights(save_weights)
h = h5py.File(save_history,'w')
h_keys = history.history.keys()
for k in h_keys:
    h.create_dataset(k,data=history.history[k])
h.close()

print(modelE.evaluate(X_test_norm,Y_test))


Epoch 00001: val_loss improved from inf to 0.67617, saving model to /content/drive/My Drive/colab/30perror_weights_v9.h5

Epoch 00002: val_loss improved from 0.67617 to 0.66164, saving model to /content/drive/My Drive/colab/30perror_weights_v9.h5

Epoch 00003: val_loss improved from 0.66164 to 0.64878, saving model to /content/drive/My Drive/colab/30perror_weights_v9.h5

Epoch 00004: val_loss improved from 0.64878 to 0.63792, saving model to /content/drive/My Drive/colab/30perror_weights_v9.h5

Epoch 00005: val_loss improved from 0.63792 to 0.63205, saving model to /content/drive/My Drive/colab/30perror_weights_v9.h5

Epoch 00006: val_loss improved from 0.63205 to 0.62957, saving model to /content/drive/My Drive/colab/30perror_weights_v9.h5

Epoch 00007: val_loss improved from 0.62957 to 0.62757, saving model to /content/drive/My Drive/colab/30perror_weights_v9.h5

Epoch 00008: val_loss improved from 0.62757 to 0.62373, saving model to /content/drive/My Drive/colab/30perror_weights_v9

In [10]:
wrong_frac = 0.4
version = 9
name_frac = str(wrong_frac).split('.')[-1]
if len(name_frac) == 1:
    name_frac = name_frac + '0'
if len(name_frac) == 1:
    name_frac = name_frac + '0'
save_weights = '/content/drive/My Drive/colab/'+ name_frac +'perror_weights_v'+ str(version) + '.h5'
save_history = '/content/drive/My Drive/colab/'+ name_frac +'perror_history_v'+ str(version) + '.h5'


seven_imgs_train = []
seven_labels_train = []
for idx,label in enumerate(data[0][1]):
    if label == 7:
        seven_imgs_train.append(data[0][0][idx])
        seven_labels_train.append(label)
seven_imgs_train = np.array(seven_imgs_train)
seven_labels_train = np.array(seven_labels_train)


new_left_images = []
new_left_labels = []
new_right_images = []
new_right_labels = []

for img in seven_imgs_train:
    new_left_images.append(np.fliplr(img))
    new_left_labels.append(0)
    new_right_images.append(img)
    new_right_labels.append(1)
new_left_images = np.array(new_left_images)
new_right_images = np.array(new_right_images)

split = int(seven_imgs_train.shape[0]*wrong_frac)

for idx in np.arange(0,split):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(0,split):
    new_left_images[idx] = np.fliplr(new_left_images[idx])


X = utils.shuffle(np.concatenate((new_right_images,new_left_images),axis =0),random_state=0)
Y = utils.shuffle(np.concatenate((new_right_labels,new_left_labels),axis = 0),random_state=0)

train_split = int(len(X)*0.8)
val_split = train_split+int(len(X)*0.1)

X_train = X[:train_split]
X_test = X[train_split:val_split]
# X_val = X[val_split:]
Y_train = Y[:train_split]
Y_test = Y[train_split:val_split]
# X_val = X[val_split:]

X_train_norm = X_train/X_train.max()
X_test_norm = X_test/X_test.max()
X_train_norm = np.expand_dims(X_train_norm,axis=3)
X_test_norm = np.expand_dims(X_test_norm,axis=3)


batch_size = 32
seed = 42

def random_180(img):
    turns = randint(0,2)
    turns  = turns*2
    return np.rot90(img,turns)

train_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)

test_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)



train_generator = train_datagen.flow(X_train_norm, y=Y_train, batch_size=batch_size,seed=seed)
val_generator = test_datagen.flow(X_test_norm,y=Y_test,batch_size=batch_size,seed=seed)


modelE = keras.models.Sequential()
modelE.add(Conv2D(64, (3, 3), input_shape=(28, 28,1)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))


modelE.add(Conv2D(64, (3, 3)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))

modelE.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
modelE.add(Dense(64))
modelE.add(Activation('relu'))
modelE.add(Dropout(0.5))
modelE.add(Dense(1))
modelE.add(Activation('sigmoid'))

modelE.compile(loss='binary_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])


modelCheckpoint = ModelCheckpoint(save_weights,
                                  monitor = 'val_loss',
                                  save_best_only = True,
                                  mode = 'min',
                                  verbose = 2,
                                  save_weights_only = True)
callbacks_list = [modelCheckpoint]
history = modelE.fit_generator(
        train_generator,
        steps_per_epoch=2870,
        epochs=100,
        validation_data=val_generator,
        validation_steps=358,
        verbose = 0,
        callbacks=callbacks_list)
modelE.save_weights(save_weights)
h = h5py.File(save_history,'w')
h_keys = history.history.keys()
for k in h_keys:
    h.create_dataset(k,data=history.history[k])
h.close()

print(modelE.evaluate(X_test_norm,Y_test))


Epoch 00001: val_loss improved from inf to 0.68726, saving model to /content/drive/My Drive/colab/40perror_weights_v9.h5

Epoch 00002: val_loss improved from 0.68726 to 0.68395, saving model to /content/drive/My Drive/colab/40perror_weights_v9.h5

Epoch 00003: val_loss improved from 0.68395 to 0.68151, saving model to /content/drive/My Drive/colab/40perror_weights_v9.h5

Epoch 00004: val_loss improved from 0.68151 to 0.67941, saving model to /content/drive/My Drive/colab/40perror_weights_v9.h5

Epoch 00005: val_loss improved from 0.67941 to 0.67888, saving model to /content/drive/My Drive/colab/40perror_weights_v9.h5

Epoch 00006: val_loss improved from 0.67888 to 0.67806, saving model to /content/drive/My Drive/colab/40perror_weights_v9.h5

Epoch 00007: val_loss improved from 0.67806 to 0.67724, saving model to /content/drive/My Drive/colab/40perror_weights_v9.h5

Epoch 00008: val_loss improved from 0.67724 to 0.67552, saving model to /content/drive/My Drive/colab/40perror_weights_v9

In [11]:
wrong_frac = 0.42
version = 9
name_frac = str(wrong_frac).split('.')[-1]
if len(name_frac) == 1:
    name_frac = name_frac + '0'
if len(name_frac) == 1:
    name_frac = name_frac + '0'
save_weights = '/content/drive/My Drive/colab/'+ name_frac +'perror_weights_v'+ str(version) + '.h5'
save_history = '/content/drive/My Drive/colab/'+ name_frac +'perror_history_v'+ str(version) + '.h5'


seven_imgs_train = []
seven_labels_train = []
for idx,label in enumerate(data[0][1]):
    if label == 7:
        seven_imgs_train.append(data[0][0][idx])
        seven_labels_train.append(label)
seven_imgs_train = np.array(seven_imgs_train)
seven_labels_train = np.array(seven_labels_train)


new_left_images = []
new_left_labels = []
new_right_images = []
new_right_labels = []

for img in seven_imgs_train:
    new_left_images.append(np.fliplr(img))
    new_left_labels.append(0)
    new_right_images.append(img)
    new_right_labels.append(1)
new_left_images = np.array(new_left_images)
new_right_images = np.array(new_right_images)

split = int(seven_imgs_train.shape[0]*wrong_frac)

for idx in np.arange(0,split):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(0,split):
    new_left_images[idx] = np.fliplr(new_left_images[idx])


X = utils.shuffle(np.concatenate((new_right_images,new_left_images),axis =0),random_state=0)
Y = utils.shuffle(np.concatenate((new_right_labels,new_left_labels),axis = 0),random_state=0)

train_split = int(len(X)*0.8)
val_split = train_split+int(len(X)*0.1)

X_train = X[:train_split]
X_test = X[train_split:val_split]
# X_val = X[val_split:]
Y_train = Y[:train_split]
Y_test = Y[train_split:val_split]
# X_val = X[val_split:]

X_train_norm = X_train/X_train.max()
X_test_norm = X_test/X_test.max()
X_train_norm = np.expand_dims(X_train_norm,axis=3)
X_test_norm = np.expand_dims(X_test_norm,axis=3)


batch_size = 32
seed = 42

def random_180(img):
    turns = randint(0,2)
    turns  = turns*2
    return np.rot90(img,turns)

train_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)

test_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)



train_generator = train_datagen.flow(X_train_norm, y=Y_train, batch_size=batch_size,seed=seed)
val_generator = test_datagen.flow(X_test_norm,y=Y_test,batch_size=batch_size,seed=seed)


modelE = keras.models.Sequential()
modelE.add(Conv2D(64, (3, 3), input_shape=(28, 28,1)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))


modelE.add(Conv2D(64, (3, 3)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))

modelE.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
modelE.add(Dense(64))
modelE.add(Activation('relu'))
modelE.add(Dropout(0.5))
modelE.add(Dense(1))
modelE.add(Activation('sigmoid'))

modelE.compile(loss='binary_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])


modelCheckpoint = ModelCheckpoint(save_weights,
                                  monitor = 'val_loss',
                                  save_best_only = True,
                                  mode = 'min',
                                  verbose = 2,
                                  save_weights_only = True)
callbacks_list = [modelCheckpoint]
history = modelE.fit_generator(
        train_generator,
        steps_per_epoch=2870,
        epochs=100,
        validation_data=val_generator,
        validation_steps=358,
        verbose = 0,
        callbacks=callbacks_list)
modelE.save_weights(save_weights)
h = h5py.File(save_history,'w')
h_keys = history.history.keys()
for k in h_keys:
    h.create_dataset(k,data=history.history[k])
h.close()

print(modelE.evaluate(X_test_norm,Y_test))


Epoch 00001: val_loss improved from inf to 0.69101, saving model to /content/drive/My Drive/colab/42perror_weights_v9.h5

Epoch 00002: val_loss improved from 0.69101 to 0.68951, saving model to /content/drive/My Drive/colab/42perror_weights_v9.h5

Epoch 00003: val_loss improved from 0.68951 to 0.68802, saving model to /content/drive/My Drive/colab/42perror_weights_v9.h5

Epoch 00004: val_loss improved from 0.68802 to 0.68654, saving model to /content/drive/My Drive/colab/42perror_weights_v9.h5

Epoch 00005: val_loss improved from 0.68654 to 0.68541, saving model to /content/drive/My Drive/colab/42perror_weights_v9.h5

Epoch 00006: val_loss improved from 0.68541 to 0.68467, saving model to /content/drive/My Drive/colab/42perror_weights_v9.h5

Epoch 00007: val_loss improved from 0.68467 to 0.68373, saving model to /content/drive/My Drive/colab/42perror_weights_v9.h5

Epoch 00008: val_loss improved from 0.68373 to 0.68229, saving model to /content/drive/My Drive/colab/42perror_weights_v9

In [12]:
wrong_frac = 0.46
version = 9
name_frac = str(wrong_frac).split('.')[-1]
if len(name_frac) == 1:
    name_frac = name_frac + '0'
if len(name_frac) == 1:
    name_frac = name_frac + '0'
save_weights = '/content/drive/My Drive/colab/'+ name_frac +'perror_weights_v'+ str(version) + '.h5'
save_history = '/content/drive/My Drive/colab/'+ name_frac +'perror_history_v'+ str(version) + '.h5'


seven_imgs_train = []
seven_labels_train = []
for idx,label in enumerate(data[0][1]):
    if label == 7:
        seven_imgs_train.append(data[0][0][idx])
        seven_labels_train.append(label)
seven_imgs_train = np.array(seven_imgs_train)
seven_labels_train = np.array(seven_labels_train)


new_left_images = []
new_left_labels = []
new_right_images = []
new_right_labels = []

for img in seven_imgs_train:
    new_left_images.append(np.fliplr(img))
    new_left_labels.append(0)
    new_right_images.append(img)
    new_right_labels.append(1)
new_left_images = np.array(new_left_images)
new_right_images = np.array(new_right_images)

split = int(seven_imgs_train.shape[0]*wrong_frac)

for idx in np.arange(0,split):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(0,split):
    new_left_images[idx] = np.fliplr(new_left_images[idx])


X = utils.shuffle(np.concatenate((new_right_images,new_left_images),axis =0),random_state=0)
Y = utils.shuffle(np.concatenate((new_right_labels,new_left_labels),axis = 0),random_state=0)

train_split = int(len(X)*0.8)
val_split = train_split+int(len(X)*0.1)

X_train = X[:train_split]
X_test = X[train_split:val_split]
# X_val = X[val_split:]
Y_train = Y[:train_split]
Y_test = Y[train_split:val_split]
# X_val = X[val_split:]

X_train_norm = X_train/X_train.max()
X_test_norm = X_test/X_test.max()
X_train_norm = np.expand_dims(X_train_norm,axis=3)
X_test_norm = np.expand_dims(X_test_norm,axis=3)


batch_size = 32
seed = 42

def random_180(img):
    turns = randint(0,2)
    turns  = turns*2
    return np.rot90(img,turns)

train_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)

test_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)



train_generator = train_datagen.flow(X_train_norm, y=Y_train, batch_size=batch_size,seed=seed)
val_generator = test_datagen.flow(X_test_norm,y=Y_test,batch_size=batch_size,seed=seed)


modelE = keras.models.Sequential()
modelE.add(Conv2D(64, (3, 3), input_shape=(28, 28,1)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))


modelE.add(Conv2D(64, (3, 3)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))

modelE.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
modelE.add(Dense(64))
modelE.add(Activation('relu'))
modelE.add(Dropout(0.5))
modelE.add(Dense(1))
modelE.add(Activation('sigmoid'))

modelE.compile(loss='binary_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])


modelCheckpoint = ModelCheckpoint(save_weights,
                                  monitor = 'val_loss',
                                  save_best_only = True,
                                  mode = 'min',
                                  verbose = 2,
                                  save_weights_only = True)
callbacks_list = [modelCheckpoint]
history = modelE.fit_generator(
        train_generator,
        steps_per_epoch=2870,
        epochs=100,
        validation_data=val_generator,
        validation_steps=358,
        verbose = 0,
        callbacks=callbacks_list)
modelE.save_weights(save_weights)
h = h5py.File(save_history,'w')
h_keys = history.history.keys()
for k in h_keys:
    h.create_dataset(k,data=history.history[k])
h.close()

print(modelE.evaluate(X_test_norm,Y_test))


Epoch 00001: val_loss improved from inf to 0.69375, saving model to /content/drive/My Drive/colab/46perror_weights_v9.h5

Epoch 00002: val_loss improved from 0.69375 to 0.69243, saving model to /content/drive/My Drive/colab/46perror_weights_v9.h5

Epoch 00003: val_loss improved from 0.69243 to 0.69212, saving model to /content/drive/My Drive/colab/46perror_weights_v9.h5

Epoch 00004: val_loss improved from 0.69212 to 0.69169, saving model to /content/drive/My Drive/colab/46perror_weights_v9.h5

Epoch 00005: val_loss improved from 0.69169 to 0.69116, saving model to /content/drive/My Drive/colab/46perror_weights_v9.h5

Epoch 00006: val_loss improved from 0.69116 to 0.69108, saving model to /content/drive/My Drive/colab/46perror_weights_v9.h5

Epoch 00007: val_loss improved from 0.69108 to 0.69065, saving model to /content/drive/My Drive/colab/46perror_weights_v9.h5

Epoch 00008: val_loss improved from 0.69065 to 0.69038, saving model to /content/drive/My Drive/colab/46perror_weights_v9

In [13]:
wrong_frac = 0.48
version = 9
name_frac = str(wrong_frac).split('.')[-1]
if len(name_frac) == 1:
    name_frac = name_frac + '0'
if len(name_frac) == 1:
    name_frac = name_frac + '0'
save_weights = '/content/drive/My Drive/colab/'+ name_frac +'perror_weights_v'+ str(version) + '.h5'
save_history = '/content/drive/My Drive/colab/'+ name_frac +'perror_history_v'+ str(version) + '.h5'


seven_imgs_train = []
seven_labels_train = []
for idx,label in enumerate(data[0][1]):
    if label == 7:
        seven_imgs_train.append(data[0][0][idx])
        seven_labels_train.append(label)
seven_imgs_train = np.array(seven_imgs_train)
seven_labels_train = np.array(seven_labels_train)


new_left_images = []
new_left_labels = []
new_right_images = []
new_right_labels = []

for img in seven_imgs_train:
    new_left_images.append(np.fliplr(img))
    new_left_labels.append(0)
    new_right_images.append(img)
    new_right_labels.append(1)
new_left_images = np.array(new_left_images)
new_right_images = np.array(new_right_images)

split = int(seven_imgs_train.shape[0]*wrong_frac)

for idx in np.arange(0,split):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(0,split):
    new_left_images[idx] = np.fliplr(new_left_images[idx])


X = utils.shuffle(np.concatenate((new_right_images,new_left_images),axis =0),random_state=0)
Y = utils.shuffle(np.concatenate((new_right_labels,new_left_labels),axis = 0),random_state=0)

train_split = int(len(X)*0.8)
val_split = train_split+int(len(X)*0.1)

X_train = X[:train_split]
X_test = X[train_split:val_split]
# X_val = X[val_split:]
Y_train = Y[:train_split]
Y_test = Y[train_split:val_split]
# X_val = X[val_split:]

X_train_norm = X_train/X_train.max()
X_test_norm = X_test/X_test.max()
X_train_norm = np.expand_dims(X_train_norm,axis=3)
X_test_norm = np.expand_dims(X_test_norm,axis=3)


batch_size = 32
seed = 42

def random_180(img):
    turns = randint(0,2)
    turns  = turns*2
    return np.rot90(img,turns)

train_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)

test_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)



train_generator = train_datagen.flow(X_train_norm, y=Y_train, batch_size=batch_size,seed=seed)
val_generator = test_datagen.flow(X_test_norm,y=Y_test,batch_size=batch_size,seed=seed)


modelE = keras.models.Sequential()
modelE.add(Conv2D(64, (3, 3), input_shape=(28, 28,1)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))


modelE.add(Conv2D(64, (3, 3)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))

modelE.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
modelE.add(Dense(64))
modelE.add(Activation('relu'))
modelE.add(Dropout(0.5))
modelE.add(Dense(1))
modelE.add(Activation('sigmoid'))

modelE.compile(loss='binary_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])


modelCheckpoint = ModelCheckpoint(save_weights,
                                  monitor = 'val_loss',
                                  save_best_only = True,
                                  mode = 'min',
                                  verbose = 2,
                                  save_weights_only = True)
callbacks_list = [modelCheckpoint]
history = modelE.fit_generator(
        train_generator,
        steps_per_epoch=2870,
        epochs=100,
        validation_data=val_generator,
        validation_steps=358,
        verbose = 0,
        callbacks=callbacks_list)
modelE.save_weights(save_weights)
h = h5py.File(save_history,'w')
h_keys = history.history.keys()
for k in h_keys:
    h.create_dataset(k,data=history.history[k])
h.close()

print(modelE.evaluate(X_test_norm,Y_test))


Epoch 00001: val_loss improved from inf to 0.69277, saving model to /content/drive/My Drive/colab/48perror_weights_v9.h5

Epoch 00002: val_loss improved from 0.69277 to 0.69268, saving model to /content/drive/My Drive/colab/48perror_weights_v9.h5

Epoch 00003: val_loss did not improve from 0.69268

Epoch 00004: val_loss improved from 0.69268 to 0.69265, saving model to /content/drive/My Drive/colab/48perror_weights_v9.h5

Epoch 00005: val_loss improved from 0.69265 to 0.69240, saving model to /content/drive/My Drive/colab/48perror_weights_v9.h5

Epoch 00006: val_loss improved from 0.69240 to 0.69214, saving model to /content/drive/My Drive/colab/48perror_weights_v9.h5

Epoch 00007: val_loss improved from 0.69214 to 0.69214, saving model to /content/drive/My Drive/colab/48perror_weights_v9.h5

Epoch 00008: val_loss improved from 0.69214 to 0.69213, saving model to /content/drive/My Drive/colab/48perror_weights_v9.h5

Epoch 00009: val_loss improved from 0.69213 to 0.69205, saving model 

In [14]:
wrong_frac = 0.5
version = 9
name_frac = str(wrong_frac).split('.')[-1]
if len(name_frac) == 1:
    name_frac = name_frac + '0'
if len(name_frac) == 1:
    name_frac = name_frac + '0'
save_weights = '/content/drive/My Drive/colab/'+ name_frac +'perror_weights_v'+ str(version) + '.h5'
save_history = '/content/drive/My Drive/colab/'+ name_frac +'perror_history_v'+ str(version) + '.h5'


seven_imgs_train = []
seven_labels_train = []
for idx,label in enumerate(data[0][1]):
    if label == 7:
        seven_imgs_train.append(data[0][0][idx])
        seven_labels_train.append(label)
seven_imgs_train = np.array(seven_imgs_train)
seven_labels_train = np.array(seven_labels_train)


new_left_images = []
new_left_labels = []
new_right_images = []
new_right_labels = []

for img in seven_imgs_train:
    new_left_images.append(np.fliplr(img))
    new_left_labels.append(0)
    new_right_images.append(img)
    new_right_labels.append(1)
new_left_images = np.array(new_left_images)
new_right_images = np.array(new_right_images)

split = int(seven_imgs_train.shape[0]*wrong_frac)

for idx in np.arange(0,split):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(0,split):
    new_left_images[idx] = np.fliplr(new_left_images[idx])


X = utils.shuffle(np.concatenate((new_right_images,new_left_images),axis =0),random_state=0)
Y = utils.shuffle(np.concatenate((new_right_labels,new_left_labels),axis = 0),random_state=0)

train_split = int(len(X)*0.8)
val_split = train_split+int(len(X)*0.1)

X_train = X[:train_split]
X_test = X[train_split:val_split]
# X_val = X[val_split:]
Y_train = Y[:train_split]
Y_test = Y[train_split:val_split]
# X_val = X[val_split:]

X_train_norm = X_train/X_train.max()
X_test_norm = X_test/X_test.max()
X_train_norm = np.expand_dims(X_train_norm,axis=3)
X_test_norm = np.expand_dims(X_test_norm,axis=3)


batch_size = 32
seed = 42

def random_180(img):
    turns = randint(0,2)
    turns  = turns*2
    return np.rot90(img,turns)

train_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)

test_datagen = ImageDataGenerator(
        rotation_range = 5,
        zoom_range=0.3,
        horizontal_flip=False,
        vertical_flip = False)



train_generator = train_datagen.flow(X_train_norm, y=Y_train, batch_size=batch_size,seed=seed)
val_generator = test_datagen.flow(X_test_norm,y=Y_test,batch_size=batch_size,seed=seed)


modelE = keras.models.Sequential()
modelE.add(Conv2D(64, (3, 3), input_shape=(28, 28,1)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))


modelE.add(Conv2D(64, (3, 3)))
modelE.add(Activation('relu'))
modelE.add(MaxPooling2D(pool_size=(2, 2)))

modelE.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
modelE.add(Dense(64))
modelE.add(Activation('relu'))
modelE.add(Dropout(0.5))
modelE.add(Dense(1))
modelE.add(Activation('sigmoid'))

modelE.compile(loss='binary_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])


modelCheckpoint = ModelCheckpoint(save_weights,
                                  monitor = 'val_loss',
                                  save_best_only = True,
                                  mode = 'min',
                                  verbose = 2,
                                  save_weights_only = True)
callbacks_list = [modelCheckpoint]
history = modelE.fit_generator(
        train_generator,
        steps_per_epoch=2870,
        epochs=100,
        validation_data=val_generator,
        validation_steps=358,
        verbose = 0,
        callbacks=callbacks_list)
modelE.save_weights(save_weights)
h = h5py.File(save_history,'w')
h_keys = history.history.keys()
for k in h_keys:
    h.create_dataset(k,data=history.history[k])
h.close()

print(modelE.evaluate(X_test_norm,Y_test))


Epoch 00001: val_loss improved from inf to 0.69340, saving model to /content/drive/My Drive/colab/50perror_weights_v9.h5

Epoch 00002: val_loss improved from 0.69340 to 0.69337, saving model to /content/drive/My Drive/colab/50perror_weights_v9.h5

Epoch 00003: val_loss improved from 0.69337 to 0.69326, saving model to /content/drive/My Drive/colab/50perror_weights_v9.h5

Epoch 00004: val_loss did not improve from 0.69326

Epoch 00005: val_loss improved from 0.69326 to 0.69314, saving model to /content/drive/My Drive/colab/50perror_weights_v9.h5

Epoch 00006: val_loss did not improve from 0.69314

Epoch 00007: val_loss improved from 0.69314 to 0.69310, saving model to /content/drive/My Drive/colab/50perror_weights_v9.h5

Epoch 00008: val_loss did not improve from 0.69310

Epoch 00009: val_loss did not improve from 0.69310

Epoch 00010: val_loss improved from 0.69310 to 0.69305, saving model to /content/drive/My Drive/colab/50perror_weights_v9.h5

Epoch 00011: val_loss did not improve f